In [55]:
import pandas as pd
import pyodbc
connection_string = ("DRIVER={SQL Server};"
"SERVER=USDF11V0954;"
"DATABASE=Actuarial_AH;"
"Trust_Connection=yes;")

conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [56]:
med = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE DRUG_NAME_PREFERRED IN ('Wegovy', 'Saxenda', 'Zepbound')
    AND SERVICE_DATE >= '2022-01-01'
    AND SERVICE_DATE < '2025-01-01'
)
SELECT M.MEDICAL_CLAIM_ID, 
       M.MEMBER_ID, 
       M.PAID_DATE, 
	   YEAR(M.PAID_DATE) AS PAID_YEAR,
       M.SERVICE_DATE,
YEAR(M.SERVICE_DATE) AS SERVICE_YEAR,
       M.MEDICAL_PAID_AMOUNT, 
       M.IS_TELEMEDICINE, 
       M.IS_ER_AVOIDABLE, 
M.DIAGNOSIS_DESC_ICD10_1 AS PRIMARY_DX,
       M.PROCEDURE_DESC, 
       M.ARTTOS_V2_L1, 
       M.ARTTOS_V2_L3, 
M.DX_IS_CHRONIC,
       M.ICD10_CHAPTER, 
       M.ICD10_CATEGORY, 
       M.ICD10_SECTION, 
       M.MEG_EPISODE_DESCRIPTION, 
       M.OP_SURG_INC,
       M.IS_PCP_VISIT, 
       M.INCLUDED_SPECIALIST, 
       M.ER_VISIT_FLAG, 
       M.IS_URGENT_CARE_VISIT, 
       M.IS_PREVENTIVE_VISIT, 
       M.IP_ADMIT_INC
FROM [Actuarial_AH].[dbo].[SN_Medical] M
JOIN 
Actuarial_AH.dbo.SN_Member MB ON M.MEMBER_ID = MB.MEMBER_ID
WHERE M.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND MB.MONTH_KEY = '2024-12-01' 
AND MB.MEDICAL_ENROLLMENT_STATUS = 3
AND MB.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND MB.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
    HAVING COUNT(DISTINCT PRESCRIPTION_YEAR) = 3
	)
"""
med = pd.read_sql(med, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_14224\403233069.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  med = pd.read_sql(med, conn)


In [57]:
rx = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE DRUG_NAME_PREFERRED IN ('Wegovy', 'Saxenda', 'Zepbound')
    AND SERVICE_DATE BETWEEN '2022-01-01' AND '2024-12-31'
)
SELECT R.RX_CLAIM_ID, 
       R.MEMBER_ID, 
       R.PAID_DATE,
       R.DAYS,
       R.SERVICE_DATE, 
       R.DRUG_NAME_GENERIC,
	   R.DRUG_NAME_PREFERRED,
	   R.RX_PAID_AMOUNT,
	   R.RX_SCRIPT_COUNT,
	   R.ART_DRUGGRPS_L1,
	   R.ART_DRUGGRPS_L2,
	   YEAR(R.PAID_DATE) AS PAID_YEAR,
YEAR(R.SERVICE_DATE) AS SERVICE_YEAR
FROM [Actuarial_AH].[dbo].[SN_Rx] R
JOIN 
Actuarial_AH.dbo.SN_Member M ON R.MEMBER_ID = M.MEMBER_ID
WHERE R.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND M.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
    HAVING COUNT(DISTINCT PRESCRIPTION_YEAR) = 3
	)
"""
rx = pd.read_sql(rx, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_14224\1756823093.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rx = pd.read_sql(rx, conn)


In [58]:
member = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE DRUG_NAME_PREFERRED IN ('Wegovy', 'Saxenda', 'Zepbound')
    AND SERVICE_DATE >= '2022-01-01'
    AND SERVICE_DATE < '2025-01-01'
)
SELECT M.MEMBER_ID,
       M.MEMBER_STATUS,
       M.HOME_STATE,
	   CASE 
    WHEN M.MEMBER_RELATIONSHIP = 0 THEN 'Subscriber'
    WHEN M.MEMBER_RELATIONSHIP = 1 THEN 'Spouse'
    WHEN M.MEMBER_RELATIONSHIP = 2 THEN 'Dependent'
    WHEN M.MEMBER_RELATIONSHIP = 3 THEN 'Domestic Partner'
    WHEN M.MEMBER_RELATIONSHIP = 4 THEN 'Unknown'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS MEMBER_RELATIONSHIP,
CASE 
        WHEN M.GENDER = 0 THEN 'Female'
        WHEN M.GENDER = 1 THEN 'Male'
        WHEN M.GENDER = 2 THEN 'Unknown'
        ELSE 'Unknown' 
    END AS GENDER,
    DATEDIFF(YEAR, CONVERT(DATE, M.BIRTH_YEAR_MO + '-01'), GETDATE()) - 
    CASE 
        WHEN MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > MONTH(GETDATE()) 
             OR (MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) = MONTH(GETDATE()) 
                 AND DAY(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > DAY(GETDATE())) 
        THEN 1 
        ELSE 0 
    END AS Age,
	CASE 
    WHEN M.BIRTH_YEAR >= 1946 AND M.BIRTH_YEAR < 1965 THEN 'Baby Boomers'
	WHEN M.BIRTH_YEAR >= 1965 AND M.BIRTH_YEAR < 1981 THEN 'Generation X'
	WHEN M.BIRTH_YEAR >= 1981 AND M.BIRTH_YEAR < 1997 THEN 'Millenials'
	WHEN M.BIRTH_YEAR >= 1997 AND M.BIRTH_YEAR < 2012 THEN 'Generation Z'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS GENERATIONS
FROM [Actuarial_AH].[dbo].[SN_MEMBER] M
WHERE M.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
    HAVING COUNT(DISTINCT PRESCRIPTION_YEAR) = 3
	)
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'


"""
member = pd.read_sql(member, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_14224\1575462883.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  member = pd.read_sql(member, conn)


In [59]:
med

,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,...,ICD10_CATEGORY,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,3526721.0,mma-1df328471ad84755af20862fceebe7d9,2021-01-01,2021,2020-07-14,2020,1.54,1.0,0.0,"Lumbago with sciatica, right side",...,Other dorsopathies,Dorsalgia,Intervertebral Disc Disorders: Lumbar and Lumb...,0,0,1,0,0,0,0
1,3554590.0,mma-1df328471ad84755af20862fceebe7d9,2021-01-01,2021,2020-07-14,2020,-1.24,0.0,0.0,"Lumbago with sciatica, right side",...,Other dorsopathies,Dorsalgia,Intervertebral Disc Disorders: Lumbar and Lumb...,0,0,0,0,0,0,0
2,3554590.0,mma-1df328471ad84755af20862fceebe7d9,2021-01-01,2021,2020-07-14,2020,-1.24,0.0,0.0,"Lumbago with sciatica, right side",...,Other dorsopathies,Dorsalgia,Intervertebral Disc Disorders: Lumbar and Lumb...,0,0,0,0,0,0,0
3,3554590.0,mma-1df328471ad84755af20862fceebe7d9,2021-01-01,2021,2020-07-14,2020,-1.23,0.0,0.0,"Lumbago with sciatica, right side",...,Other dorsopathies,Dorsalgia,Intervertebral Disc Disorders: Lumbar and Lumb...,0,0,0,0,0,0,0
4,7409997.0,mma-54e4a63c58a94557b0006e9008f84e58,2021-01-01,2021,2020-11-12,2020,66.00,1.0,0.0,"Adjustment disorder, unspecified",...,"Anxiety, dissociative, stress-related, somatof...","Reaction to severe stress, and adjustment diso...",Depression,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187022,7267041.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-12-11,2024,0.00,0.0,1.0,Pain in right hip,...,Other joint disorders,"Other joint disorder, not elsewhere classified","Fracture, Dislocation, or Sprain: Hip or Pelvis",0,0,0,0,0,0,0
187023,3066260.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-11-26,2024,0.00,0.0,1.0,Other low back pain,...,Other dorsopathies,Dorsalgia,Intervertebral Disc Disorders: Lumbar and Lumb...,0,0,0,0,0,0,0
187024,3066260.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-11-26,2024,31.81,0.0,1.0,Pain in left hip,...,Other joint disorders,"Other joint disorder, not elsewhere classified","Fracture, Dislocation, or Sprain: Hip or Pelvis",0,0,0,0,0,0,0
187025,3066260.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-11-26,2024,22.99,0.0,1.0,Pain in right hip,...,Other joint disorders,"Other joint disorder, not elsewhere classified","Fracture, Dislocation, or Sprain: Hip or Pelvis",0,0,0,0,0,0,0


In [60]:
rx

,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,DAYS,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR
0,4113483.0,mma-1938a618fe3143cb811290f8b38bd6cc,2021-01-02,84.0,2021-01-02,Desogestrel-Ethinyl Estradiol,Apri,23.28,84.0,Hormones,Hormonal Contraceptives,2021,2021
1,13707649.0,mma-26c50e65030c4e798cdd7244f7a78a3a,2021-01-07,30.0,2021-01-06,Pantoprazole Sodium,Pantoprazole Sodium,3.14,30.0,Gastrointestinal,Proton Pum Inhibitors,2021,2021
2,12086669.0,mma-727fdd04aed245d383ce207f6488efae,2021-01-07,90.0,2021-01-07,Sertraline HCl,Sertraline HCl,0.00,90.0,CNS Agents,Antidepressants - SSRI,2021,2021
3,12086669.0,mma-727fdd04aed245d383ce207f6488efae,2021-01-07,-90.0,2021-01-07,Sertraline HCl,Sertraline HCl,0.00,-90.0,CNS Agents,Antidepressants - SSRI,2021,2021
4,2064631.0,mma-065d185c362a42b2bb84100d232e86e6,2021-01-06,90.0,2021-01-06,Atorvastatin Calcium,Atorvastatin Calcium,73.05,90.0,Lipid Agents,HMG CoA Reductase Inhibitors,2021,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116444,10827064.0,mma-e9a5c6a46a764504b8601c3c9e882046,2024-12-31,90.0,2024-12-15,Atorvastatin Calcium,Atorvastatin Calcium,0.00,90.0,Lipid Agents,HMG CoA Reductase Inhibitors,2024,2024
116445,18053692.0,mma-b71c47eb1bb04c189244799c4c47d1d0,2024-12-31,84.0,2024-12-31,Desogestrel-Ethinyl Estradiol,Volnea,33.82,84.0,Hormones,Hormonal Contraceptives,2024,2024
116446,10827064.0,mma-e9a5c6a46a764504b8601c3c9e882046,2024-12-31,90.0,2024-12-15,Atorvastatin Calcium,Atorvastatin Calcium,0.00,90.0,Lipid Agents,HMG CoA Reductase Inhibitors,2024,2024
116447,228907.0,mma-a79a8316db394ec9ae4aa1176994ff1f,2024-12-31,28.0,2024-12-31,Tirzepatide-Weight Management,Zepbound,1029.42,2.0,CNS Agents,Weight Loss Agents,2024,2024


In [61]:
member

,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS
0,mma-5a400fb9c1934bcf87c25301710034c3,0.0,PA,Spouse,Female,58,Generation X
1,mma-2e25ed887e0b4990ad2fa1c2d91e0966,0.0,NC,Subscriber,Female,53,Generation X
2,mma-b027fbc0d58348338e0a12b5520743a1,0.0,NC,Subscriber,Female,48,Generation X
3,mma-8f71aebcee534b149fcb3a24f53d271c,0.0,NC,Subscriber,Female,45,Generation X
4,mma-6bdc0ffa0d8e406c8d5c977b9573bc60,0.0,TX,Subscriber,Female,40,Millenials
...,...,...,...,...,...,...,...
721,mma-d141003390034a5f8f93b87662a23576,0.0,PA,Spouse,Female,43,Millenials
722,mma-4a8193ed89cb490a9d052dffc42ef258,0.0,KY,Subscriber,Female,57,Generation X
723,mma-b860c502e71844cd8332ab944c68eb5c,0.0,TX,Subscriber,Female,33,Millenials
724,mma-9c36bd8aa8d440a9bd298bc8487a7c46,0.0,AZ,Subscriber,Male,63,Baby Boomers


In [62]:
member.to_csv('glp-1-member-claims.csv', index=False)
med.to_csv('med-glp-claims.csv', index=False)
rx.to_csv('rx-glp-claims.csv', index=False)

In [63]:
med = pd.read_csv('med-glp-claims.csv')
rx = pd.read_csv('rx-glp-claims.csv')
member = pd.read_csv('glp-1-member-claims.csv')

In [64]:
# Define the drugs to filter out
drugs_to_remove = ['Wegovy', 'Saxenda', 'Zepbound']

# Filter to find members with the specified drugs in 2021
members_to_remove = rx[(rx['SERVICE_YEAR'] == 2021) & (rx['DRUG_NAME_PREFERRED'].isin(drugs_to_remove))]['MEMBER_ID'].unique()

# Remove those members from the original DataFrame
rx_members_glp_22_24 = rx[~rx['MEMBER_ID'].isin(members_to_remove)]
med_members_glp_22_24 = med[~med['MEMBER_ID'].isin(members_to_remove)]
members_glp_22_24 = member[~member['MEMBER_ID'].isin(members_to_remove)]

In [65]:
# Define NDCs or names of GLP-1 drugs
glp1_drugs = ['Wegovy', 'Saxenda', 'Zepbound']  # Replace with actual NDCs for Wegovy, Zepbound, Saxenda

# Step 1: Filter for GLP-1 claims only
glp1_drugs_rx = rx_members_glp_22_24[rx_members_glp_22_24['DRUG_NAME_PREFERRED'].isin(glp1_drugs)].copy()

# Step 2: Convert fill_date to datetime
glp1_drugs_rx['SERVICE_DATE'] = pd.to_datetime(glp1_drugs_rx['SERVICE_DATE'])

# Step 3: Create a column for days covered per fill
glp1_drugs_rx['END_DATE'] = glp1_drugs_rx['SERVICE_DATE'] + pd.to_timedelta(glp1_drugs_rx['DAYS'] - 1, unit='D')
glp1_drugs_rx

,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,DAYS,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR,END_DATE
4451,3975501.0,mma-327abcd778724f35bf6696e3700ba2d4,2022-02-03,28.0,2022-02-03,Semaglutide-Weight Management,Wegovy,1268.61,2.0,CNS Agents,Weight Loss Agents,2022,2022,2022-03-02
4455,3975501.0,mma-327abcd778724f35bf6696e3700ba2d4,2022-02-03,-28.0,2022-02-03,Semaglutide-Weight Management,Wegovy,-1268.61,-2.0,CNS Agents,Weight Loss Agents,2022,2022,2022-01-05
4664,4567739.0,mma-327abcd778724f35bf6696e3700ba2d4,2022-02-12,28.0,2022-02-12,Semaglutide-Weight Management,Wegovy,1268.61,2.0,CNS Agents,Weight Loss Agents,2022,2022,2022-03-11
4680,469034.0,mma-06e49f14fe354dc3b81eaca36fd040ff,2022-02-23,28.0,2022-02-23,Semaglutide-Weight Management,Wegovy,1225.13,2.0,CNS Agents,Weight Loss Agents,2022,2022,2022-03-22
4689,7141965.0,mma-4c5c5820d8724eca9b3d34c9c18185e2,2022-02-16,28.0,2022-02-16,Semaglutide-Weight Management,Wegovy,1245.93,2.0,CNS Agents,Weight Loss Agents,2022,2022,2022-03-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116415,1849328.0,mma-d276af8503864048a053d5f7f753d000,2024-12-26,28.0,2024-12-26,Semaglutide-Weight Management,Wegovy,1310.13,3.0,CNS Agents,Weight Loss Agents,2024,2024,2025-01-22
116419,2304279.0,mma-efdcd2cf2933464bae7a6c6273ea7260,2024-12-27,28.0,2024-12-28,Tirzepatide-Weight Management,Zepbound,977.95,2.0,CNS Agents,Weight Loss Agents,2024,2024,2025-01-24
116432,10226216.0,mma-f066c4f2cd9c4c6c96be8294430faa7b,2024-12-30,28.0,2024-12-30,Semaglutide-Weight Management,Wegovy,1242.41,3.0,CNS Agents,Weight Loss Agents,2024,2024,2025-01-26
116433,2560562.0,mma-80c4f6ee7c934b888938e0e98f2b7508,2024-12-30,28.0,2024-12-30,Semaglutide-Weight Management,Wegovy,1310.13,3.0,CNS Agents,Weight Loss Agents,2024,2024,2025-01-26


In [66]:
member_pdc = glp1_drugs_rx.groupby('MEMBER_ID').agg(
    min_service_date=('SERVICE_DATE', 'min'),
    max_end_date=('END_DATE', 'max'),
    covered_days = ('DAYS', 'sum')
).reset_index()
member_pdc['Total_Days'] = (member_pdc['max_end_date'] - member_pdc['min_service_date']).dt.total_seconds() / (24 * 3600)

member_pdc['PDC'] = member_pdc['covered_days']/member_pdc['Total_Days']
member_pdc

,MEMBER_ID,min_service_date,max_end_date,covered_days,Total_Days,PDC
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,2022-07-01,2025-01-15,874.0,929.0,0.940797
1,mma-0054034112c64cd2924c4f85de9a46ca,2022-02-23,2025-03-02,1048.0,1103.0,0.950136
2,mma-010b8797194145fbb3687e60959516b1,2022-07-22,2025-01-05,320.0,898.0,0.356347
3,mma-01c846aa858d43e98167eb0669db3a4e,2022-03-15,2024-05-12,534.0,789.0,0.676806
4,mma-0281be178d4042ab83dd30b3190a3c6c,2022-08-24,2025-01-22,1410.0,882.0,1.598639
...,...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,2022-11-29,2025-01-06,1558.0,769.0,2.026008
457,mma-fdc88c2f941e45d3808795b5eb01e19e,2022-02-11,2025-03-20,934.0,1133.0,0.824360
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,2022-10-17,2025-01-07,862.0,813.0,1.060271
459,mma-ff82ab56bc48478b91ad9760b62d56e1,2022-09-26,2024-07-24,284.0,667.0,0.425787


In [67]:
member_pdc_80 = member_pdc.copy()
member_pdc_80 = member_pdc_80[member_pdc_80['PDC'] >= 0.8]
member_pdc_80

,MEMBER_ID,min_service_date,max_end_date,covered_days,Total_Days,PDC
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,2022-07-01,2025-01-15,874.0,929.0,0.940797
1,mma-0054034112c64cd2924c4f85de9a46ca,2022-02-23,2025-03-02,1048.0,1103.0,0.950136
4,mma-0281be178d4042ab83dd30b3190a3c6c,2022-08-24,2025-01-22,1410.0,882.0,1.598639
7,mma-038ef0da0fd54c2c81bfd92b13f81a3e,2022-11-10,2025-01-08,641.0,790.0,0.811392
9,mma-041ff13e5c1e4fd08b24fd0c67c8af63,2022-07-12,2025-01-06,962.0,909.0,1.058306
...,...,...,...,...,...,...
452,mma-fb5675af09a14732bb702c1feb30e1b6,2022-07-15,2025-01-05,1872.0,905.0,2.068508
456,mma-fd3a666933464a569791e983fbdb4867,2022-11-29,2025-01-06,1558.0,769.0,2.026008
457,mma-fdc88c2f941e45d3808795b5eb01e19e,2022-02-11,2025-03-20,934.0,1133.0,0.824360
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,2022-10-17,2025-01-07,862.0,813.0,1.060271


In [ ]:
pdc80 = member_pdc_80['MEMBER_ID'].unique()

# Filter to find members with the specified drugs in 2021
members_to_include = rx[rx['MEMBER_ID'].isin(pdc80)]

# Remove those members from the original DataFrame
rx_members_glp_22_24 = rx[~rx['MEMBER_ID'].isin(members_to_remove)]
med_members_glp_22_24 = med[~med['MEMBER_ID'].isin(members_to_remove)]
members_glp_22_24 = member[~member['MEMBER_ID'].isin(members_to_remove)]

In [68]:
med_members_glp_22_24

,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,...,ICD10_CATEGORY,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,3526721.0,mma-1df328471ad84755af20862fceebe7d9,2021-01-01,2021,2020-07-14,2020,1.54,1.0,0.0,"Lumbago with sciatica, right side",...,Other dorsopathies,Dorsalgia,Intervertebral Disc Disorders: Lumbar and Lumb...,0,0,1,0,0,0,0
1,3554590.0,mma-1df328471ad84755af20862fceebe7d9,2021-01-01,2021,2020-07-14,2020,-1.24,0.0,0.0,"Lumbago with sciatica, right side",...,Other dorsopathies,Dorsalgia,Intervertebral Disc Disorders: Lumbar and Lumb...,0,0,0,0,0,0,0
2,3554590.0,mma-1df328471ad84755af20862fceebe7d9,2021-01-01,2021,2020-07-14,2020,-1.24,0.0,0.0,"Lumbago with sciatica, right side",...,Other dorsopathies,Dorsalgia,Intervertebral Disc Disorders: Lumbar and Lumb...,0,0,0,0,0,0,0
3,3554590.0,mma-1df328471ad84755af20862fceebe7d9,2021-01-01,2021,2020-07-14,2020,-1.23,0.0,0.0,"Lumbago with sciatica, right side",...,Other dorsopathies,Dorsalgia,Intervertebral Disc Disorders: Lumbar and Lumb...,0,0,0,0,0,0,0
5,5700925.0,mma-64d3f45898914e7eacee663dca120f1e,2021-01-01,2021,2020-12-22,2020,347.17,0.0,1.0,Encounter for gynecological examination (gener...,...,Persons encountering health services for exami...,Encounter for other special examination withou...,Encounter for Preventive Health Services,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187022,7267041.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-12-11,2024,0.00,0.0,1.0,Pain in right hip,...,Other joint disorders,"Other joint disorder, not elsewhere classified","Fracture, Dislocation, or Sprain: Hip or Pelvis",0,0,0,0,0,0,0
187023,3066260.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-11-26,2024,0.00,0.0,1.0,Other low back pain,...,Other dorsopathies,Dorsalgia,Intervertebral Disc Disorders: Lumbar and Lumb...,0,0,0,0,0,0,0
187024,3066260.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-11-26,2024,31.81,0.0,1.0,Pain in left hip,...,Other joint disorders,"Other joint disorder, not elsewhere classified","Fracture, Dislocation, or Sprain: Hip or Pelvis",0,0,0,0,0,0,0
187025,3066260.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-11-26,2024,22.99,0.0,1.0,Pain in right hip,...,Other joint disorders,"Other joint disorder, not elsewhere classified","Fracture, Dislocation, or Sprain: Hip or Pelvis",0,0,0,0,0,0,0


In [69]:
rx_members_glp_22_24

,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,DAYS,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR
4,2064631.0,mma-065d185c362a42b2bb84100d232e86e6,2021-01-06,90.0,2021-01-06,Atorvastatin Calcium,Atorvastatin Calcium,73.05,90.0,Lipid Agents,HMG CoA Reductase Inhibitors,2021,2021
5,4203921.0,mma-1c7664ff40a54d9f9de3a968e361c1c5,2021-01-06,90.0,2021-01-07,Losartan Potassium,Losartan Potassium,0.00,90.0,Cardiac,Angiotensin Receptor Blockers,2021,2021
10,7163997.0,mma-4c5c5820d8724eca9b3d34c9c18185e2,2021-01-05,25.0,2021-01-06,Albuterol Sulfate HFA,Albuterol Sulfate HFA,17.76,6.7,Respiratory,Inhaled Beta Agonists & Combos,2021,2021
13,1487168.0,mma-15bab83022f44682a4db20abb18e9cd3,2021-01-04,5.0,2021-01-04,ALPRAZolam,ALPRAZolam,0.00,15.0,CNS Agents,Benzodiazepines,2021,2021
14,740654.0,mma-15bab83022f44682a4db20abb18e9cd3,2021-01-04,90.0,2021-01-04,Escitalopram Oxalate,Escitalopram Oxalate,55.01,90.0,CNS Agents,Antidepressants - SSRI,2021,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116443,2279674.0,mma-8058a3f66c8f4e7f9e8d729e1e7e6dde,2024-12-31,28.0,2024-12-31,Semaglutide-Weight Management,Wegovy,1255.13,3.0,CNS Agents,Weight Loss Agents,2024,2024
116444,10827064.0,mma-e9a5c6a46a764504b8601c3c9e882046,2024-12-31,90.0,2024-12-15,Atorvastatin Calcium,Atorvastatin Calcium,0.00,90.0,Lipid Agents,HMG CoA Reductase Inhibitors,2024,2024
116445,18053692.0,mma-b71c47eb1bb04c189244799c4c47d1d0,2024-12-31,84.0,2024-12-31,Desogestrel-Ethinyl Estradiol,Volnea,33.82,84.0,Hormones,Hormonal Contraceptives,2024,2024
116446,10827064.0,mma-e9a5c6a46a764504b8601c3c9e882046,2024-12-31,90.0,2024-12-15,Atorvastatin Calcium,Atorvastatin Calcium,0.00,90.0,Lipid Agents,HMG CoA Reductase Inhibitors,2024,2024


In [70]:
med_agg = med_members_glp_22_24.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='MEDICAL_PAID_AMOUNT', aggfunc='sum')
med_agg.reset_index(inplace=True)
med_agg.columns.name = None
med_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in med_agg.columns]
med_agg.rename(columns={'2021': 'Med_2021', '2022': 'Med_2022', '2023': 'Med_2023', '2024': 'Med_2024'}, inplace=True)
med_agg1 = med_agg
med_agg1

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,708.74,1069.66,13211.02,7120.77
1,mma-0054034112c64cd2924c4f85de9a46ca,8338.34,739.25,780.80,2225.00
2,mma-010b8797194145fbb3687e60959516b1,2493.20,621.00,79.93,734.63
3,mma-01c846aa858d43e98167eb0669db3a4e,951.22,496.56,449.34,330.20
4,mma-0281be178d4042ab83dd30b3190a3c6c,3834.97,2604.28,3228.46,14857.68
...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,43039.79,64591.43,32627.37,35547.06
457,mma-fdc88c2f941e45d3808795b5eb01e19e,1687.41,1501.48,1604.11,2188.31
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,45.33,28.19,0.00,204.80
459,mma-ff82ab56bc48478b91ad9760b62d56e1,11481.20,2310.52,1079.37,14244.44


In [71]:

# Define inflation rates (as factors)
inflation_factors = {
    2021: 1.175, 
    2022: 1.117,   
    2023: 1.069, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in inflation_factors.items():
    column_name = f'Med_{year}'
    med_agg[column_name] = med_agg[column_name] * factor
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,832.76950,1194.81022,14122.58038,7120.77
1,mma-0054034112c64cd2924c4f85de9a46ca,9797.54950,825.74225,834.67520,2225.00
2,mma-010b8797194145fbb3687e60959516b1,2929.51000,693.65700,85.44517,734.63
3,mma-01c846aa858d43e98167eb0669db3a4e,1117.68350,554.65752,480.34446,330.20
4,mma-0281be178d4042ab83dd30b3190a3c6c,4506.08975,2908.98076,3451.22374,14857.68
...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,50571.75325,72148.62731,34878.65853,35547.06
457,mma-fdc88c2f941e45d3808795b5eb01e19e,1982.70675,1677.15316,1714.79359,2188.31
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,53.26275,31.48823,0.00000,204.80
459,mma-ff82ab56bc48478b91ad9760b62d56e1,13490.41000,2580.85084,1153.84653,14244.44


In [72]:
med_agg['Med_Delta'] = med_agg['Med_2024'] - med_agg['Med_2021']
med_agg['Med_%_Change'] =(( med_agg['Med_2024'] - med_agg['Med_2021'])/ med_agg['Med_2021']) * 100
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,832.76950,1194.81022,14122.58038,7120.77,6288.00050,755.070941
1,mma-0054034112c64cd2924c4f85de9a46ca,9797.54950,825.74225,834.67520,2225.00,-7572.54950,-77.290240
2,mma-010b8797194145fbb3687e60959516b1,2929.51000,693.65700,85.44517,734.63,-2194.88000,-74.923110
3,mma-01c846aa858d43e98167eb0669db3a4e,1117.68350,554.65752,480.34446,330.20,-787.48350,-70.456753
4,mma-0281be178d4042ab83dd30b3190a3c6c,4506.08975,2908.98076,3451.22374,14857.68,10351.59025,229.724458
...,...,...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,50571.75325,72148.62731,34878.65853,35547.06,-15024.69325,-29.709655
457,mma-fdc88c2f941e45d3808795b5eb01e19e,1982.70675,1677.15316,1714.79359,2188.31,205.60325,10.369827
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,53.26275,31.48823,0.00000,204.80,151.53725,284.508873
459,mma-ff82ab56bc48478b91ad9760b62d56e1,13490.41000,2580.85084,1153.84653,14244.44,754.03000,5.589378


In [73]:
med_agg.fillna(0, inplace=True)

In [74]:
med_agg.isnull().sum()

MEMBER_ID       0
Med_2021        0
Med_2022        0
Med_2023        0
Med_2024        0
Med_Delta       0
Med_%_Change    0
dtype: int64

In [75]:
import numpy as np

# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in med_agg.iterrows():
    costs = np.array([row['Med_2021'], row['Med_2022'], row['Med_2023'], row['Med_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
med_agg['Med_Slope'] = slope_list
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,832.76950,1194.81022,14122.58038,7120.77,6288.00050,755.070941,3179.177166
1,mma-0054034112c64cd2924c4f85de9a46ca,9797.54950,825.74225,834.67520,2225.00,-7572.54950,-77.290240,-2270.871555
2,mma-010b8797194145fbb3687e60959516b1,2929.51000,693.65700,85.44517,734.63,-2194.88000,-74.923110,-719.285183
3,mma-01c846aa858d43e98167eb0669db3a4e,1117.68350,554.65752,480.34446,330.20,-787.48350,-70.456753,-243.676356
4,mma-0281be178d4042ab83dd30b3190a3c6c,4506.08975,2908.98076,3451.22374,14857.68,10351.59025,229.724458,3159.701373
...,...,...,...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,50571.75325,72148.62731,34878.65853,35547.06,-15024.69325,-29.709655,-8234.404853
457,mma-fdc88c2f941e45d3808795b5eb01e19e,1982.70675,1677.15316,1714.79359,2188.31,205.60325,10.369827,65.445018
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,53.26275,31.48823,0.00000,204.80,151.53725,284.508873,42.312352
459,mma-ff82ab56bc48478b91ad9760b62d56e1,13490.41000,2580.85084,1153.84653,14244.44,754.03000,5.589378,83.508569


In [76]:
rx_agg = rx_members_glp_22_24.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='RX_PAID_AMOUNT', aggfunc='sum')
rx_agg.reset_index(inplace=True)
rx_agg.columns.name = None
rx_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in rx_agg.columns]
rx_agg.rename(columns={'2021': 'Rx_2021', '2022': 'Rx_2022', '2023': 'Rx_2023', '2024': 'Rx_2024'}, inplace=True)
rx_agg1 = rx_agg
rx_agg1

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,7596.54,15354.04,20686.86,13584.34
1,mma-0054034112c64cd2924c4f85de9a46ca,NaN,14183.54,14311.33,18501.11
2,mma-010b8797194145fbb3687e60959516b1,12.74,3812.79,3781.08,6361.50
3,mma-01c846aa858d43e98167eb0669db3a4e,1809.09,13926.68,9073.36,5200.05
4,mma-0281be178d4042ab83dd30b3190a3c6c,791.36,5048.15,15017.36,17215.45
...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,2679.97,6276.88,24392.79,15549.13
457,mma-fdc88c2f941e45d3808795b5eb01e19e,114.09,6248.18,15425.84,18245.69
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,3082.47,10946.63,19182.19,18446.81
459,mma-ff82ab56bc48478b91ad9760b62d56e1,321.53,3053.49,1877.35,11416.44


In [77]:
# Define inflation rates (as factors)
rx_inflation_factors = {
    2021: 1.31, 
    2022: 1.20,   
    2023: 1.103, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in rx_inflation_factors.items():
    column_name = f'Rx_{year}'
    rx_agg[column_name] = rx_agg[column_name] * factor
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,9951.4674,18424.848,22817.60658,13584.34
1,mma-0054034112c64cd2924c4f85de9a46ca,NaN,17020.248,15785.39699,18501.11
2,mma-010b8797194145fbb3687e60959516b1,16.6894,4575.348,4170.53124,6361.50
3,mma-01c846aa858d43e98167eb0669db3a4e,2369.9079,16712.016,10007.91608,5200.05
4,mma-0281be178d4042ab83dd30b3190a3c6c,1036.6816,6057.780,16564.14808,17215.45
...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,3510.7607,7532.256,26905.24737,15549.13
457,mma-fdc88c2f941e45d3808795b5eb01e19e,149.4579,7497.816,17014.70152,18245.69
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,4038.0357,13135.956,21157.95557,18446.81
459,mma-ff82ab56bc48478b91ad9760b62d56e1,421.2043,3664.188,2070.71705,11416.44


In [78]:
rx_agg.fillna(0, inplace=True)
rx_agg.isnull().sum()

MEMBER_ID    0
Rx_2021      0
Rx_2022      0
Rx_2023      0
Rx_2024      0
dtype: int64

In [79]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [80]:
rx_agg['Rx_Delta'] = rx_agg['Rx_2024'] - rx_agg['Rx_2021']
rx_agg['Rx_%_Change'] =((rx_agg['Rx_2024'] - rx_agg['Rx_2021'])/ rx_agg['Rx_2021']) * 100
# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in rx_agg.iterrows():
    costs = np.array([row['Rx_2021'], row['Rx_2022'], row['Rx_2023'], row['Rx_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
rx_agg['Rx_Slope'] = slope_list
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,9951.47,18424.85,22817.61,13584.34,3632.87,36.51,1529.14
1,mma-0054034112c64cd2924c4f85de9a46ca,0.00,17020.25,15785.40,18501.11,18501.11,inf,5426.85
2,mma-010b8797194145fbb3687e60959516b1,16.69,4575.35,4170.53,6361.50,6344.81,38017.01,1862.96
3,mma-01c846aa858d43e98167eb0669db3a4e,2369.91,16712.02,10007.92,5200.05,2830.14,119.42,178.63
4,mma-0281be178d4042ab83dd30b3190a3c6c,1036.68,6057.78,16564.15,17215.45,16178.77,1560.63,5904.27
...,...,...,...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,3510.76,7532.26,26905.25,15549.13,12038.37,342.90,5548.81
457,mma-fdc88c2f941e45d3808795b5eb01e19e,149.46,7497.82,17014.70,18245.69,18096.23,12107.91,6380.56
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,4038.04,13135.96,21157.96,18446.81,14408.77,356.83,5124.83
459,mma-ff82ab56bc48478b91ad9760b62d56e1,421.20,3664.19,2070.72,11416.44,10995.24,2610.43,3139.22


In [81]:
rx_agg.replace(np.inf, 0, inplace=True)
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,9951.47,18424.85,22817.61,13584.34,3632.87,36.51,1529.14
1,mma-0054034112c64cd2924c4f85de9a46ca,0.00,17020.25,15785.40,18501.11,18501.11,0.00,5426.85
2,mma-010b8797194145fbb3687e60959516b1,16.69,4575.35,4170.53,6361.50,6344.81,38017.01,1862.96
3,mma-01c846aa858d43e98167eb0669db3a4e,2369.91,16712.02,10007.92,5200.05,2830.14,119.42,178.63
4,mma-0281be178d4042ab83dd30b3190a3c6c,1036.68,6057.78,16564.15,17215.45,16178.77,1560.63,5904.27
...,...,...,...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,3510.76,7532.26,26905.25,15549.13,12038.37,342.90,5548.81
457,mma-fdc88c2f941e45d3808795b5eb01e19e,149.46,7497.82,17014.70,18245.69,18096.23,12107.91,6380.56
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,4038.04,13135.96,21157.96,18446.81,14408.77,356.83,5124.83
459,mma-ff82ab56bc48478b91ad9760b62d56e1,421.20,3664.19,2070.72,11416.44,10995.24,2610.43,3139.22


In [82]:
member_mrg = members_glp_22_24.merge(med_agg, on='MEMBER_ID')
member_mrg = member_mrg.merge(rx_agg, on='MEMBER_ID')                                     
member_mrg                                    

,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,...,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-5a400fb9c1934bcf87c25301710034c3,0.00,PA,Spouse,Female,58,Generation X,5783.55,11169.72,10984.51,...,7882.82,136.30,2346.32,63018.44,91263.11,52811.56,69490.32,6471.88,10.27,-1903.59
1,mma-2e25ed887e0b4990ad2fa1c2d91e0966,0.00,NC,Subscriber,Female,53,Generation X,32448.94,4684.03,12931.39,...,-25683.37,-79.15,-6880.27,1063.89,8594.32,19333.53,13359.87,12295.98,1155.76,4762.72
2,mma-b027fbc0d58348338e0a12b5520743a1,0.00,NC,Subscriber,Female,48,Generation X,743.06,1762.83,1853.81,...,-271.74,-36.57,-72.42,0.00,2192.41,7835.17,12179.12,12179.12,0.00,4218.01
3,mma-d9fa4b33e14b458992bf0617961ac987,0.00,NC,Subscriber,Female,53,Generation X,1175.70,4011.46,2656.13,...,-292.82,-24.91,-223.38,0.00,2196.13,22039.20,15271.61,15271.61,0.00,6565.79
4,mma-a809caefdd8242f3ac89858b2ef2a9c0,0.00,NC,Subscriber,Female,56,Generation X,6175.82,2379.46,9456.50,...,-2206.45,-35.73,45.77,936.58,21588.54,17339.17,15705.83,14769.25,1576.93,4005.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,mma-041ff13e5c1e4fd08b24fd0c67c8af63,0.00,NC,Subscriber,Female,45,Generation X,5188.04,69740.30,4861.18,...,6264.86,120.76,-4608.45,345.16,9334.48,18574.66,11003.33,10658.17,3087.90,4121.47
457,mma-06e49f14fe354dc3b81eaca36fd040ff,0.00,NC,Spouse,Male,51,Generation X,451.04,5114.60,4853.21,...,1735.46,384.77,494.50,77.19,17838.62,17524.42,14644.65,14567.46,18873.39,4338.82
458,mma-4a8193ed89cb490a9d052dffc42ef258,0.00,KY,Subscriber,Female,57,Generation X,2199.55,725.65,844.00,...,-1287.54,-58.54,-374.43,9739.05,13377.96,14854.35,11039.27,1300.22,13.35,537.71
459,mma-b860c502e71844cd8332ab944c68eb5c,0.00,TX,Subscriber,Female,33,Millenials,4460.62,1562.09,8551.75,...,-3824.06,-85.73,-448.25,99.23,3120.52,6774.45,14362.64,14263.41,14373.73,4644.42


In [83]:
member1 = member_mrg.drop(columns=['MEMBER_STATUS'])
member1

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-5a400fb9c1934bcf87c25301710034c3,PA,Spouse,Female,58,Generation X,5783.55,11169.72,10984.51,13666.37,7882.82,136.30,2346.32,63018.44,91263.11,52811.56,69490.32,6471.88,10.27,-1903.59
1,mma-2e25ed887e0b4990ad2fa1c2d91e0966,NC,Subscriber,Female,53,Generation X,32448.94,4684.03,12931.39,6765.57,-25683.37,-79.15,-6880.27,1063.89,8594.32,19333.53,13359.87,12295.98,1155.76,4762.72
2,mma-b027fbc0d58348338e0a12b5520743a1,NC,Subscriber,Female,48,Generation X,743.06,1762.83,1853.81,471.32,-271.74,-36.57,-72.42,0.00,2192.41,7835.17,12179.12,12179.12,0.00,4218.01
3,mma-d9fa4b33e14b458992bf0617961ac987,NC,Subscriber,Female,53,Generation X,1175.70,4011.46,2656.13,882.88,-292.82,-24.91,-223.38,0.00,2196.13,22039.20,15271.61,15271.61,0.00,6565.79
4,mma-a809caefdd8242f3ac89858b2ef2a9c0,NC,Subscriber,Female,56,Generation X,6175.82,2379.46,9456.50,3969.37,-2206.45,-35.73,45.77,936.58,21588.54,17339.17,15705.83,14769.25,1576.93,4005.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,mma-041ff13e5c1e4fd08b24fd0c67c8af63,NC,Subscriber,Female,45,Generation X,5188.04,69740.30,4861.18,11452.90,6264.86,120.76,-4608.45,345.16,9334.48,18574.66,11003.33,10658.17,3087.90,4121.47
457,mma-06e49f14fe354dc3b81eaca36fd040ff,NC,Spouse,Male,51,Generation X,451.04,5114.60,4853.21,2186.50,1735.46,384.77,494.50,77.19,17838.62,17524.42,14644.65,14567.46,18873.39,4338.82
458,mma-4a8193ed89cb490a9d052dffc42ef258,KY,Subscriber,Female,57,Generation X,2199.55,725.65,844.00,912.01,-1287.54,-58.54,-374.43,9739.05,13377.96,14854.35,11039.27,1300.22,13.35,537.71
459,mma-b860c502e71844cd8332ab944c68eb5c,TX,Subscriber,Female,33,Millenials,4460.62,1562.09,8551.75,636.56,-3824.06,-85.73,-448.25,99.23,3120.52,6774.45,14362.64,14263.41,14373.73,4644.42


In [84]:
member1.HOME_STATE.value_counts()

HOME_STATE
NC    197
PA     85
TX     29
MI     22
NJ     21
NY     17
GA     12
VA      9
FL      8
IL      7
KY      7
OH      6
CA      5
KS      4
DE      4
TN      4
IN      3
MN      3
MO      3
CO      2
WI      2
MA      2
SC      1
CT      1
ID      1
OK      1
NH      1
MS      1
WA      1
MD      1
OR      1
Name: count, dtype: int64

In [85]:
# Define the states to keep
states_to_keep = ['NC', 'PA', 'TX', 'MI', 'NJ', 'NY', 'GA']
member1['HOME_STATE'] = member1['HOME_STATE'].apply(lambda x: x if x in states_to_keep else 'OTHER')
member1

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-5a400fb9c1934bcf87c25301710034c3,PA,Spouse,Female,58,Generation X,5783.55,11169.72,10984.51,13666.37,7882.82,136.30,2346.32,63018.44,91263.11,52811.56,69490.32,6471.88,10.27,-1903.59
1,mma-2e25ed887e0b4990ad2fa1c2d91e0966,NC,Subscriber,Female,53,Generation X,32448.94,4684.03,12931.39,6765.57,-25683.37,-79.15,-6880.27,1063.89,8594.32,19333.53,13359.87,12295.98,1155.76,4762.72
2,mma-b027fbc0d58348338e0a12b5520743a1,NC,Subscriber,Female,48,Generation X,743.06,1762.83,1853.81,471.32,-271.74,-36.57,-72.42,0.00,2192.41,7835.17,12179.12,12179.12,0.00,4218.01
3,mma-d9fa4b33e14b458992bf0617961ac987,NC,Subscriber,Female,53,Generation X,1175.70,4011.46,2656.13,882.88,-292.82,-24.91,-223.38,0.00,2196.13,22039.20,15271.61,15271.61,0.00,6565.79
4,mma-a809caefdd8242f3ac89858b2ef2a9c0,NC,Subscriber,Female,56,Generation X,6175.82,2379.46,9456.50,3969.37,-2206.45,-35.73,45.77,936.58,21588.54,17339.17,15705.83,14769.25,1576.93,4005.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,mma-041ff13e5c1e4fd08b24fd0c67c8af63,NC,Subscriber,Female,45,Generation X,5188.04,69740.30,4861.18,11452.90,6264.86,120.76,-4608.45,345.16,9334.48,18574.66,11003.33,10658.17,3087.90,4121.47
457,mma-06e49f14fe354dc3b81eaca36fd040ff,NC,Spouse,Male,51,Generation X,451.04,5114.60,4853.21,2186.50,1735.46,384.77,494.50,77.19,17838.62,17524.42,14644.65,14567.46,18873.39,4338.82
458,mma-4a8193ed89cb490a9d052dffc42ef258,OTHER,Subscriber,Female,57,Generation X,2199.55,725.65,844.00,912.01,-1287.54,-58.54,-374.43,9739.05,13377.96,14854.35,11039.27,1300.22,13.35,537.71
459,mma-b860c502e71844cd8332ab944c68eb5c,TX,Subscriber,Female,33,Millenials,4460.62,1562.09,8551.75,636.56,-3824.06,-85.73,-448.25,99.23,3120.52,6774.45,14362.64,14263.41,14373.73,4644.42


In [86]:
med.columns

Index(['MEDICAL_CLAIM_ID', 'MEMBER_ID', 'PAID_DATE', 'PAID_YEAR',
       'SERVICE_DATE', 'SERVICE_YEAR', 'MEDICAL_PAID_AMOUNT',
       'IS_TELEMEDICINE', 'IS_ER_AVOIDABLE', 'PRIMARY_DX', 'PROCEDURE_DESC',
       'ARTTOS_V2_L1', 'ARTTOS_V2_L3', 'DX_IS_CHRONIC', 'ICD10_CHAPTER',
       'ICD10_CATEGORY', 'ICD10_SECTION', 'MEG_EPISODE_DESCRIPTION',
       'OP_SURG_INC', 'IS_PCP_VISIT', 'INCLUDED_SPECIALIST', 'ER_VISIT_FLAG',
       'IS_URGENT_CARE_VISIT', 'IS_PREVENTIVE_VISIT', 'IP_ADMIT_INC'],
      dtype='object')

In [87]:
med_util = med.groupby('MEMBER_ID').agg({
    'DX_IS_CHRONIC': 'max',
    'ER_VISIT_FLAG': 'sum',
    'IS_PCP_VISIT': 'sum',
    'IS_URGENT_CARE_VISIT': 'sum',
    'IS_PREVENTIVE_VISIT': 'sum',
    'IP_ADMIT_INC': 'sum'
}).reset_index()
med_util

,MEMBER_ID,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,1.00,0,8,0,5,0
1,mma-0054034112c64cd2924c4f85de9a46ca,1.00,0,8,1,4,0
2,mma-010b8797194145fbb3687e60959516b1,1.00,2,2,1,0,0
3,mma-01c846aa858d43e98167eb0669db3a4e,1.00,0,6,1,6,0
4,mma-0281be178d4042ab83dd30b3190a3c6c,1.00,43,18,9,5,0
...,...,...,...,...,...,...,...
721,mma-fe17629a213c48c299541abcf3098a08,1.00,4,60,0,2,0
722,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,1.00,0,4,0,1,0
723,mma-ff1781243560419d986a286a5cae8c89,1.00,2,6,4,3,0
724,mma-ff82ab56bc48478b91ad9760b62d56e1,1.00,13,16,17,3,0


In [89]:
episodes = med.ICD10_CHAPTER.value_counts()
episodes.head(10)

ICD10_CHAPTER
Factors influencing health status and contact with health services                         34854
Diseases of the musculoskeletal system and connective tissue                               31694
Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified    22463
Endocrine, nutritional and metabolic diseases                                              19369
Diseases of the genitourinary system                                                       10485
Mental, Behavioral and Neurodevelopmental disorders                                         9994
Diseases of the circulatory system                                                          8479
Diseases of the digestive system                                                            7855
Diseases of the nervous system                                                              7606
Diseases of the respiratory system                                                          6290
Name: count, dty

In [90]:
episodes_cost = med.groupby('ICD10_CHAPTER')['MEDICAL_PAID_AMOUNT'].sum().reset_index()
episodes_cost.sort_values(by='MEDICAL_PAID_AMOUNT', ascending=False)

,ICD10_CHAPTER,MEDICAL_PAID_AMOUNT
9,Diseases of the musculoskeletal system and con...,5277105.73
15,Factors influencing health status and contact ...,3041389.10
20,"Symptoms, signs and abnormal clinical and labo...",2460776.19
5,Diseases of the digestive system,2237161.53
8,Diseases of the genitourinary system,1618215.07
13,"Endocrine, nutritional and metabolic diseases",1541442.60
10,Diseases of the nervous system,1468785.59
18,Neoplasms,1337388.94
16,"Injury, poisoning and certain other consequenc...",1332167.24
4,Diseases of the circulatory system,1289525.47


In [91]:
episodes_to_keep = ['Diseases of the musculoskeletal system and connective tissue', 'Neoplasms', 'Diseases of the digestive system', 
                    'Diseases of the circulatory system', 'Diseases of the circulatory system', 'Diseases of the genitourinary system',
                   'Diseases of the nervous system', 'Mental, Behavioral and Neurodevelopmental disorders']
med1 = med
med1['ICD10_CHAPTER'] = med['ICD10_CHAPTER'].apply(lambda x: x if x in episodes_to_keep else 'OTHER')

In [92]:
med_meg = pd.pivot_table(
    med1,
    values='MEDICAL_PAID_AMOUNT',
    index='MEMBER_ID',
    columns='ICD10_CHAPTER',
    aggfunc='sum',
    fill_value=0
)
med_meg.reset_index()
med_meg.columns.name = None
med_meg.drop(columns='OTHER', inplace=True)
med_meg

,Diseases of the circulatory system,Diseases of the digestive system,Diseases of the genitourinary system,Diseases of the musculoskeletal system and connective tissue,Diseases of the nervous system,"Mental, Behavioral and Neurodevelopmental disorders",Neoplasms
MEMBER_ID,,,,,,,
mma-0049f2985b9f413a8cf5ad2f79c95d4b,260.53,0.00,1790.54,12640.50,30.85,1180.76,73.88
mma-0054034112c64cd2924c4f85de9a46ca,550.71,5866.48,351.29,0.00,0.00,0.00,0.00
mma-010b8797194145fbb3687e60959516b1,0.00,0.00,0.00,90.98,0.00,2323.44,0.00
mma-01c846aa858d43e98167eb0669db3a4e,0.00,0.00,0.00,0.00,51.15,263.51,69.36
mma-0281be178d4042ab83dd30b3190a3c6c,490.72,14131.24,299.39,2242.95,100.66,-190.00,0.00
...,...,...,...,...,...,...,...
mma-fe17629a213c48c299541abcf3098a08,8313.49,1225.43,34508.64,69458.56,315200.07,11173.74,171.28
mma-fe87330ad1d34f5bb4ee0bba91cf61c7,28.19,0.00,0.00,0.00,0.00,0.00,0.00
mma-ff1781243560419d986a286a5cae8c89,930.95,0.00,0.00,0.00,0.00,0.00,0.00


In [93]:
med_var = member1.merge(med_util, on='MEMBER_ID')
med_var = med_var.merge(med_meg, on='MEMBER_ID')
med_var.columns= med_var.columns.str.replace(' ', '_')
med_var

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,...,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC,Diseases_of_the_circulatory_system,Diseases_of_the_digestive_system,Diseases_of_the_genitourinary_system,Diseases_of_the_musculoskeletal_system_and_connective_tissue,Diseases_of_the_nervous_system,"Mental,_Behavioral_and_Neurodevelopmental_disorders",Neoplasms
0,mma-5a400fb9c1934bcf87c25301710034c3,PA,Spouse,Female,58,Generation X,5783.55,11169.72,10984.51,13666.37,...,2,6,0,0.00,137.84,2048.62,6526.45,4145.57,768.79,0.00
1,mma-2e25ed887e0b4990ad2fa1c2d91e0966,NC,Subscriber,Female,53,Generation X,32448.94,4684.03,12931.39,6765.57,...,13,24,0,2125.92,99.20,5.83,26560.68,7610.95,621.18,0.00
2,mma-b027fbc0d58348338e0a12b5520743a1,NC,Subscriber,Female,48,Generation X,743.06,1762.83,1853.81,471.32,...,2,1,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,mma-d9fa4b33e14b458992bf0617961ac987,NC,Subscriber,Female,53,Generation X,1175.70,4011.46,2656.13,882.88,...,2,2,0,1640.38,0.00,0.00,564.04,1413.91,27.70,0.00
4,mma-a809caefdd8242f3ac89858b2ef2a9c0,NC,Subscriber,Female,56,Generation X,6175.82,2379.46,9456.50,3969.37,...,7,3,0,0.00,2102.30,0.00,7860.53,1891.86,42.65,307.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,mma-041ff13e5c1e4fd08b24fd0c67c8af63,NC,Subscriber,Female,45,Generation X,5188.04,69740.30,4861.18,11452.90,...,3,4,21,57.25,0.00,6251.39,1693.88,125.00,1107.21,940.64
457,mma-06e49f14fe354dc3b81eaca36fd040ff,NC,Spouse,Male,51,Generation X,451.04,5114.60,4853.21,2186.50,...,1,3,0,169.40,0.00,0.00,2616.84,241.71,0.00,2099.96
458,mma-4a8193ed89cb490a9d052dffc42ef258,OTHER,Subscriber,Female,57,Generation X,2199.55,725.65,844.00,912.01,...,0,9,0,0.00,0.00,75.42,148.77,0.00,0.00,0.00
459,mma-b860c502e71844cd8332ab944c68eb5c,TX,Subscriber,Female,33,Millenials,4460.62,1562.09,8551.75,636.56,...,1,3,0,0.00,0.00,510.20,206.62,158.16,0.00,586.78


In [94]:
med_var.columns

Index(['MEMBER_ID', 'HOME_STATE', 'MEMBER_RELATIONSHIP', 'GENDER', 'Age',
       'GENERATIONS', 'Med_2021', 'Med_2022', 'Med_2023', 'Med_2024',
       'Med_Delta', 'Med_%_Change', 'Med_Slope', 'Rx_2021', 'Rx_2022',
       'Rx_2023', 'Rx_2024', 'Rx_Delta', 'Rx_%_Change', 'Rx_Slope',
       'DX_IS_CHRONIC', 'ER_VISIT_FLAG', 'IS_PCP_VISIT',
       'IS_URGENT_CARE_VISIT', 'IS_PREVENTIVE_VISIT', 'IP_ADMIT_INC',
       'Diseases_of_the_circulatory_system',
       'Diseases_of_the_digestive_system',
       'Diseases_of_the_genitourinary_system',
       'Diseases_of_the_musculoskeletal_system_and_connective_tissue',
       'Diseases_of_the_nervous_system',
       'Mental,_Behavioral_and_Neurodevelopmental_disorders', 'Neoplasms'],
      dtype='object')

In [95]:
rx_members_glp_22_24.describe()

,RX_CLAIM_ID,DAYS,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,PAID_YEAR,SERVICE_YEAR
count,69622.00,68800.00,69622.00,69622.00,69622.00,69622.00
mean,5670892.90,36.58,294.78,38.63,2022.66,2022.64
std,4798771.88,35.57,1347.46,117.75,1.07,1.08
min,110.00,-91.00,-37470.72,-4000.00,2021.00,2019.00
25%,1644333.25,28.00,0.00,3.00,2022.00,2022.00
50%,3987786.00,30.00,7.28,30.00,2023.00,2023.00
75%,9341614.00,60.00,112.27,60.00,2024.00,2024.00
max,21089586.00,175.00,37470.72,6000.00,2024.00,2025.00


In [96]:
member_pdc.describe()

,min_service_date,max_end_date,covered_days,Total_Days,PDC
count,461,454,461.00,454.00,454.00
mean,2022-07-06 03:16:47.375271168,2024-12-14 21:11:53.656387584,815.59,893.41,0.92
min,2022-01-04 00:00:00,2024-02-21 00:00:00,0.00,454.00,0.03
25%,2022-03-24 00:00:00,2024-12-20 00:00:00,544.00,787.25,0.63
50%,2022-06-24 00:00:00,2025-01-07 00:00:00,740.00,896.50,0.86
75%,2022-10-26 00:00:00,2025-01-19 00:00:00,982.00,1007.50,1.06
max,2022-12-31 00:00:00,2025-03-23 00:00:00,3494.00,1140.00,4.93
std,NaN,NaN,482.00,134.56,0.51


In [97]:
med_var.describe()

,Age,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,...,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC,Diseases_of_the_circulatory_system,Diseases_of_the_digestive_system,Diseases_of_the_genitourinary_system,Diseases_of_the_musculoskeletal_system_and_connective_tissue,Diseases_of_the_nervous_system,"Mental,_Behavioral_and_Neurodevelopmental_disorders",Neoplasms
count,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,...,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00
mean,49.11,10785.83,8428.90,8103.09,9513.89,-1554.05,inf,-414.16,4030.60,12071.98,...,1.85,4.00,3.67,1800.23,3290.98,1876.91,6595.02,1811.03,969.33,1592.80
std,9.57,28876.91,17172.10,15900.76,30679.53,40012.57,NaN,12270.73,16040.31,18195.96,...,2.90,3.78,10.18,8770.60,12394.07,4883.69,31737.77,13696.99,2999.93,7236.54
min,19.00,-6733.78,-2675.94,0.00,-3113.33,-351945.16,-218.99,-106061.39,-35797.51,0.00,...,0.00,0.00,0.00,-1252.66,0.00,-6572.25,-242.78,0.00,-190.00,0.00
25%,43.00,927.12,1266.58,1105.60,1153.71,-2486.06,-60.81,-968.58,82.82,3229.66,...,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,49.00,2591.38,3068.68,2975.91,2912.37,152.06,7.08,31.01,353.31,7369.97,...,1.00,3.00,0.00,38.93,0.00,186.41,416.44,0.00,0.00,0.00
75%,56.00,7357.73,7854.88,7862.89,8883.64,2871.99,234.71,1322.50,1281.63,15020.11,...,2.00,5.00,0.00,612.00,1054.14,986.03,2397.00,397.50,460.21,245.27
max,76.00,357120.96,222048.77,186327.16,566726.05,565812.16,inf,169296.90,201096.74,223585.97,...,21.00,38.00,74.00,99842.85,199730.50,35391.03,536353.62,273354.75,31947.72,119677.08


In [98]:
costs = med_var[['MEMBER_ID', 'HOME_STATE', 'MEMBER_RELATIONSHIP', 'GENDER', 'Age',
       'GENERATIONS', 'Med_2021', 'Med_2022', 'Med_2023', 'Med_2024',
       'Med_Delta', 'Med_%_Change', 'Med_Slope', 'Rx_2021', 'Rx_2022',
       'Rx_2023', 'Rx_2024', 'Rx_Delta', 'Rx_%_Change', 'Rx_Slope']]
costs

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-5a400fb9c1934bcf87c25301710034c3,PA,Spouse,Female,58,Generation X,5783.55,11169.72,10984.51,13666.37,7882.82,136.30,2346.32,63018.44,91263.11,52811.56,69490.32,6471.88,10.27,-1903.59
1,mma-2e25ed887e0b4990ad2fa1c2d91e0966,NC,Subscriber,Female,53,Generation X,32448.94,4684.03,12931.39,6765.57,-25683.37,-79.15,-6880.27,1063.89,8594.32,19333.53,13359.87,12295.98,1155.76,4762.72
2,mma-b027fbc0d58348338e0a12b5520743a1,NC,Subscriber,Female,48,Generation X,743.06,1762.83,1853.81,471.32,-271.74,-36.57,-72.42,0.00,2192.41,7835.17,12179.12,12179.12,0.00,4218.01
3,mma-d9fa4b33e14b458992bf0617961ac987,NC,Subscriber,Female,53,Generation X,1175.70,4011.46,2656.13,882.88,-292.82,-24.91,-223.38,0.00,2196.13,22039.20,15271.61,15271.61,0.00,6565.79
4,mma-a809caefdd8242f3ac89858b2ef2a9c0,NC,Subscriber,Female,56,Generation X,6175.82,2379.46,9456.50,3969.37,-2206.45,-35.73,45.77,936.58,21588.54,17339.17,15705.83,14769.25,1576.93,4005.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,mma-041ff13e5c1e4fd08b24fd0c67c8af63,NC,Subscriber,Female,45,Generation X,5188.04,69740.30,4861.18,11452.90,6264.86,120.76,-4608.45,345.16,9334.48,18574.66,11003.33,10658.17,3087.90,4121.47
457,mma-06e49f14fe354dc3b81eaca36fd040ff,NC,Spouse,Male,51,Generation X,451.04,5114.60,4853.21,2186.50,1735.46,384.77,494.50,77.19,17838.62,17524.42,14644.65,14567.46,18873.39,4338.82
458,mma-4a8193ed89cb490a9d052dffc42ef258,OTHER,Subscriber,Female,57,Generation X,2199.55,725.65,844.00,912.01,-1287.54,-58.54,-374.43,9739.05,13377.96,14854.35,11039.27,1300.22,13.35,537.71
459,mma-b860c502e71844cd8332ab944c68eb5c,TX,Subscriber,Female,33,Millenials,4460.62,1562.09,8551.75,636.56,-3824.06,-85.73,-448.25,99.23,3120.52,6774.45,14362.64,14263.41,14373.73,4644.42


In [99]:
costs.describe()

,Age,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
count,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00
mean,49.11,10785.83,8428.90,8103.09,9513.89,-1554.05,inf,-414.16,4030.60,12071.98,17114.74,15865.22,11834.62,13919.00,4054.66
std,9.57,28876.91,17172.10,15900.76,30679.53,40012.57,NaN,12270.73,16040.31,18195.96,17275.50,18398.69,18506.06,87465.91,6022.53
min,19.00,-6733.78,-2675.94,0.00,-3113.33,-351945.16,-218.99,-106061.39,-35797.51,0.00,39.83,307.20,-178299.60,-146.54,-48086.39
25%,43.00,927.12,1266.58,1105.60,1153.71,-2486.06,-60.81,-968.58,82.82,3229.66,9224.75,8872.36,6904.86,234.86,2328.96
50%,49.00,2591.38,3068.68,2975.91,2912.37,152.06,7.08,31.01,353.31,7369.97,15280.25,12927.53,11249.63,1821.01,3890.98
75%,56.00,7357.73,7854.88,7862.89,8883.64,2871.99,234.71,1322.50,1281.63,15020.11,18511.48,16237.32,14922.30,6774.62,5168.75
max,76.00,357120.96,222048.77,186327.16,566726.05,565812.16,inf,169296.90,201096.74,223585.97,170115.73,206102.47,203204.41,1732629.77,70188.75


average med+rx net per member, normalized for 2024 = $11,834-1,554 = $10,280. Since 2021, employers have spent $10,280 average per member on those that have prescriptions for Wegovy, Saxenda or Zepbound for 3 years

In [100]:
med_var.to_csv('glp1_members.csv', index=False)

In [101]:
drugs = rx.groupby(['ART_DRUGGRPS_L1', 'ART_DRUGGRPS_L2', 'DRUG_NAME_PREFERRED'])['RX_PAID_AMOUNT'].sum().reset_index()
drugs.sort_values(by='RX_PAID_AMOUNT', ascending=False, inplace=True)
drugs

,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT
303,CNS Agents,Weight Loss Agents,Wegovy,17251767.28
302,CNS Agents,Weight Loss Agents,Saxenda,4358296.11
305,CNS Agents,Weight Loss Agents,Zepbound,1593951.74
86,CNS Agents,Anti-Rheumatics/Immune Modulators,Humira (2 Pen),1133602.61
1076,Topical Agents,Psoriasis Agents,Stelara,1072764.97
...,...,...,...,...
465,Endocrine,Thyroid Agents,Nature-Throid,-0.62
213,CNS Agents,NSAID Analgesics,Duexis,-20.70
528,Gastrointestinal,Proton Pum Inhibitors,Omeprazole-Sodium Bicarbonate,-5403.49
221,CNS Agents,NSAID Analgesics,Ketoprofen,-7348.34


In [45]:
med.to_csv('GLP-1_Medical.csv', index=False)